In [1]:
import requests

In [2]:
base_url = "https://books.toscrape.com/index.html"
page = requests.get(base_url)

if page.status_code == 200:
  print("Site is ready for scraping")
else:
  print("Site is not ready for scraping")

Site is ready for scraping


In [3]:
!pip install beautifulsoup4


[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from bs4 import BeautifulSoup

In [6]:
soup = BeautifulSoup(markup=page.content,features="html.parser")
books = soup.find_all(name="li",class_="col-xs-6 col-sm-4 col-md-3 col-lg-3")
len(books)

20

In [9]:
book = books[0]
book

<li class="col-xs-6 col-sm-4 col-md-3 col-lg-3">
<article class="product_pod">
<div class="image_container">
<a href="catalogue/a-light-in-the-attic_1000/index.html"><img alt="A Light in the Attic" class="thumbnail" src="media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg"/></a>
</div>
<p class="star-rating Three">
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
</p>
<h3><a href="catalogue/a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a></h3>
<div class="product_price">
<p class="price_color">£51.77</p>
<p class="instock availability">
<i class="icon-ok"></i>
    
        In stock
    
</p>
<form>
<button class="btn btn-primary btn-block" data-loading-text="Adding..." type="submit">Add to basket</button>
</form>
</div>
</article>
</li>

In [10]:
book_url = book.findChild(name="a").get("href")
book_url

C:\Users\HARSH\AppData\Local\Temp\ipykernel_4988\526733149.py:1: DeprecationWarning: Call to deprecated method findChild. (Replaced by find) -- Deprecated since version 3.0.0.
  book_url = book.findChild(name="a").get("href")


'catalogue/a-light-in-the-attic_1000/index.html'

In [11]:
from urllib.parse import urljoin
final_url = urljoin(base_url,book_url)
final_url

'https://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'

In [13]:
book_info = requests.get(final_url).content
book_soup = BeautifulSoup(markup=book_info,features="html.parser")

In [14]:
book_soup


<!DOCTYPE html>

<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-us"> <!--<![endif]-->
<head>
<title>
    A Light in the Attic | Books to Scrape - Sandbox
</title>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="24th Jun 2016 09:29" name="created"/>
<meta content="
    It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love th It's hard to imagine a world without A Light in the Attic. Th

In [15]:
book_table = book_soup.find_all(name="tr")
len(book_table)

7

In [16]:
book_data = {}
for row in book_table:
  key = row.find(name="th").getText()
  value = row.find(name="td").getText()
  book_data[key] = value

book_data

{'UPC': 'a897fe39b1053632',
 'Product Type': 'Books',
 'Price (excl. tax)': '£51.77',
 'Price (incl. tax)': '£51.77',
 'Tax': '£0.00',
 'Availability': 'In stock (22 available)',
 'Number of reviews': '0'}

In [19]:
def scrape_book(book_url):
  book_info = requests.get(book_url).content
  book_soup = BeautifulSoup(markup=book_info, features="html.parser")

  book_data = {}

  # getting name
  name = book_soup.find(name="h1").getText()
  book_data['name'] = name

  # getting other data
  book_table_data = book_soup.find_all(name="tr")
  for row in book_table_data:
    key = row.find(name="th").getText()
    value = row.find(name="td").getText()
    book_data[key] = value

  # let's also keep the url of book in final result
  book_data['url'] = book_url
  return book_data

In [20]:
# let's test this
scrape_book("https://books.toscrape.com/catalogue/its-only-the-himalayas_981/index.html")

{'name': "It's Only the Himalayas",
 'UPC': 'a22124811bfa8350',
 'Product Type': 'Books',
 'Price (excl. tax)': '£45.17',
 'Price (incl. tax)': '£45.17',
 'Tax': '£0.00',
 'Availability': 'In stock (19 available)',
 'Number of reviews': '0',
 'url': 'https://books.toscrape.com/catalogue/its-only-the-himalayas_981/index.html'}

In [21]:
page_count = 1
data = []

while True:
  page_url = f"https://books.toscrape.com/catalogue/page-{page_count}.html"
  status = requests.get(page_url).status_code

  # break the loop if we exceed the total page count
  if status == 404:
    break

  page_data = scrape_book(page_url)
  data.extend(page_data) # do not use .append() since the function returns a list
  print(f"Page: {page_count} is SUCCESSFULLY scraped")

  page_count += 1

Page: 1 is SUCCESSFULLY scraped
Page: 2 is SUCCESSFULLY scraped
Page: 3 is SUCCESSFULLY scraped
Page: 4 is SUCCESSFULLY scraped
Page: 5 is SUCCESSFULLY scraped
Page: 6 is SUCCESSFULLY scraped
Page: 7 is SUCCESSFULLY scraped
Page: 8 is SUCCESSFULLY scraped
Page: 9 is SUCCESSFULLY scraped
Page: 10 is SUCCESSFULLY scraped
Page: 11 is SUCCESSFULLY scraped
Page: 12 is SUCCESSFULLY scraped
Page: 13 is SUCCESSFULLY scraped
Page: 14 is SUCCESSFULLY scraped
Page: 15 is SUCCESSFULLY scraped
Page: 16 is SUCCESSFULLY scraped
Page: 17 is SUCCESSFULLY scraped
Page: 18 is SUCCESSFULLY scraped
Page: 19 is SUCCESSFULLY scraped
Page: 20 is SUCCESSFULLY scraped
Page: 21 is SUCCESSFULLY scraped
Page: 22 is SUCCESSFULLY scraped
Page: 23 is SUCCESSFULLY scraped
Page: 24 is SUCCESSFULLY scraped
Page: 25 is SUCCESSFULLY scraped
Page: 26 is SUCCESSFULLY scraped
Page: 27 is SUCCESSFULLY scraped
Page: 28 is SUCCESSFULLY scraped
Page: 29 is SUCCESSFULLY scraped
Page: 30 is SUCCESSFULLY scraped
Page: 31 is SUCCESS

In [22]:
book_data = {}
for row in book_table:
  key = row.find(name="th").getText()
  value = row.find(name="td").getText()
  book_data[key] = value

book_data

{'UPC': 'a897fe39b1053632',
 'Product Type': 'Books',
 'Price (excl. tax)': '£51.77',
 'Price (incl. tax)': '£51.77',
 'Tax': '£0.00',
 'Availability': 'In stock (22 available)',
 'Number of reviews': '0'}

In [23]:
#Lets Fix the data : 'Price (excl. tax)': '£51.77'

float(book_data['Price (excl. tax)'][1:])

51.77

In [24]:
#'Availability': 'In stock (22 available)' --> (22,In stock)
qty_available = int(book_data['Availability'].split("(")[-1][:-1].split()[0])
is_available = book_data['Availability'].split("(")[0].strip()
qty_available,is_available

(22, 'In stock')

In [25]:
def fix(item):
  item['Price (excl. tax)'] = float(item['Price (excl. tax)'][1:])
  item['Price (incl. tax)'] = float(item['Price (incl. tax)'][1:])
  item['Tax'] = float(item['Tax'][1:])
  availability = item.pop('Availability')
  item['is_available'] = True if availability.split("(")[0].strip() == 'In stock' else False
  item['quantity_available'] = int(availability.split("(")[-1][:-1].split()[0])
  return item

In [26]:
fix(book_data)

{'UPC': 'a897fe39b1053632',
 'Product Type': 'Books',
 'Price (excl. tax)': 51.77,
 'Price (incl. tax)': 51.77,
 'Tax': 0.0,
 'Number of reviews': '0',
 'is_available': True,
 'quantity_available': 22}